In [1]:
import numpy as np
from sympy import Matrix, zoo

class SimplexSolver:
    def __init__(self, filename):
        self.A, self.b, self.c, self.z = self._read_csv(filename)
        self.b_org = self.b.copy()
        self.epsilon = 0.5  # Equivalent to pow(2, -1)

    def _read_csv(self, filename):
        '''
        Reads the input matrix A, vectors b and c, and initial point z from a CSV file.
        '''
        try:
            with open(filename, 'r') as input_file:
                # Read the first line to extract vector z
                line = input_file.readline()
                if not line:
                    raise ValueError('Input file is empty or does not contain initial point z.')
                z = list(map(float, line.strip().split(',')))
                z = np.asarray(z[:-1], dtype=float)

                # Read the second line to extract vector c
                line = input_file.readline()
                if not line:
                    raise ValueError('Input file does not contain cost vector c.')
                c = list(map(float, line.strip().split(',')))
                c = np.asarray(c[:-1], dtype=float)

                # Read all the remaining lines to extract matrix A and vector b
                A, b = [], []
                for line in input_file:
                    l = list(map(float, line.strip().split(',')))
                    A.append(l[:-1])
                    b.append(l[-1])
                if not A or not b:
                    raise ValueError('Input file does not contain matrix A and vector b.')

                A = np.asarray(A, dtype=float)
                b = np.asarray(b, dtype=float)

            return A, b, c, z
        except FileNotFoundError:
            raise FileNotFoundError(f'File "{filename}" not found.')
        except Exception as e:
            raise ValueError(f'Error reading file "{filename}": {e}')

    def _test_feasibility(self, z):
        '''
        Checks whether a point z is a feasible solution for the system or not.
        '''
        return np.all(np.matmul(self.A, z) <= self.b)

    def _split_rows(self, z):
        '''
        Splits the matrices A and b into sets of tight and untight rows with respect to a given point z.
        '''
        A1, A2, b1, b2 = [], [], [], []
        Az = np.dot(self.A, z)
        for i in range(len(Az)):
            if np.isclose(np.dot(self.A[i], z), self.b[i]):
                A1.append(self.A[i])
                b1.append(self.b[i])
            else:
                A2.append(self.A[i])
                b2.append(self.b[i])
        return np.array(A1), np.array(b1), np.array(A2), np.array(b2)

    def _test_vertex(self, z):
        '''
        Checks whether a point z is a vertex of the feasible region or not.
        '''
        A1, _, _, _ = self._split_rows(z)
        rank_A = np.linalg.matrix_rank(self.A)
        num_tight_rows = A1.shape[0]
        return rank_A <= num_tight_rows

    def _move(self, z):
        '''
        Moves the feasible point towards a vertex of the feasible region by one step size.
        '''
        try:
            m, n = self.A.shape
            A1, _, A2, b2 = self._split_rows(z)
            A1 = A1.reshape((A1.shape[0], n))

            # Compute the direction to move the feasible point
            nullspace = np.array(Matrix(A1).nullspace())
            if nullspace.size == 0:
                raise ValueError('Nullspace of A1 is empty; cannot move towards vertex.')
            scale = np.random.randint(1, 4, nullspace.shape[0]).reshape(nullspace.shape[0], 1, 1)
            vector = np.array(np.sum(nullspace * scale, axis=0)).astype(float)
            u = vector / np.linalg.norm(vector)

            # Compute the step size 'alpha'
            temp = np.dot(A2, u)
            temp[np.isclose(temp, 0)] = 1e-32  # Avoid division by zero
            alpha_values = (b2 - np.dot(A2, z)) / temp.T
            alpha_values = alpha_values[0]
            alpha_values = np.array([a for a in alpha_values if not ((a > 1e6) or (-a) > 1e6)])
            if alpha_values.size == 0:
                raise ValueError('No valid alpha found; cannot move towards vertex.')
            alpha = alpha_values[np.argmin(np.abs(alpha_values.copy()))]

            # Update the feasible point by one step size
            u = u.T
            z_new = z + alpha * u
            z_new = z_new[0]

            return z_new
        except Exception as e:
            raise RuntimeError(f'Error in moving towards vertex: {e}')

    def _test_degeneracy(self, z):
        '''
        Checks whether a vertex z of the feasible region is degenerate.
        '''
        A1, _, _, _ = self._split_rows(z)
        return np.linalg.matrix_rank(self.A) < A1.shape[0]

    def _remove_degeneracy(self, b):
        '''
        Removes degeneracy from the feasible region by adding powers of a small positive number to vector b.
        '''
        add = self.epsilon
        for i in range(len(b)):
            b[i] += add
            add *= self.epsilon
        return b

    def _test_optimum(self, z):
        '''
        Checks whether a vertex z is an optimum vertex or not.
        '''
        try:
            A1, _, _, _ = self._split_rows(z)
            beta = np.dot(np.linalg.inv(A1.T), self.c.T)
            return np.all(beta >= 0)
        except np.linalg.LinAlgError:
            raise np.linalg.LinAlgError('Matrix A1.T is singular; cannot compute beta.')
        except Exception as e:
            raise RuntimeError(f'Error in testing for optimum: {e}')

    def _optimize(self, z):
        '''
        Optimizes the cost function by moving to the next optimum vertex.
        '''
        try:
            A1, _, A2, b2 = self._split_rows(z)

            # Compute the direction to move in order to reach a more optimum vertex
            A1_inverse = np.linalg.inv(A1)
            A1_inverse = np.transpose(A1_inverse)
            u = None
            for x in A1_inverse:
                if np.dot(-x, self.c.T) > 0:
                    u = x.T
                    break
            if u is None:
                raise ValueError('No direction found to optimize the cost function.')

            # Compute the step size 'alpha'
            temp = np.dot(A2, u)
            temp[np.isclose(temp, 0)] = 1e-16  # Avoid division by zero
            alpha_values = (np.dot(A2, z) - b2) / temp
            alpha_values = [x for x in alpha_values if x != zoo]
            if all([a < 0 for a in alpha_values]):
                return z, False
            alpha = min([x for x in alpha_values if x >= 0])

            # Update the vertex
            z_new = z - alpha * u

            return z_new, True
        except np.linalg.LinAlgError:
            raise np.linalg.LinAlgError('Matrix A1 is singular; cannot compute inverse.')
        except Exception as e:
            raise RuntimeError(f'Error in optimization step: {e}')

    def solve(self):
        '''
        Driver function which calls the subroutines according to the Simplex Algorithm.
        '''
        try:
            # Step 1: Check for feasibility of the Initial Point
            if not self._test_feasibility(self.z):
                raise ValueError(f'Initial point {self.z} is not feasible.')

            solved = False

            while not solved:
                # Step 2: Move towards the first vertex
                while not self._test_vertex(self.z):
                    self.z = self._move(self.z)

                # Test for degeneracy of the vertex
                if self._test_degeneracy(self.z):
                    self.b = self._remove_degeneracy(self.b_org.copy())
                    self.epsilon *= 2
                    continue

                # Print the value of the Cost function at this first vertex
                print(f'Vertex {self.z} --> Cost: {np.dot(self.z, self.c)}')

                # Step 3: March through every vertex and check for optimum along with degeneracy
                solution = True
                degenerate = False
                while solution and (not self._test_optimum(self.z)):
                    self.z, solution = self._optimize(self.z)
                    if not solution:
                        raise ValueError('No solution found during optimization.')
                    elif self._test_degeneracy(self.z):
                        self.b = self._remove_degeneracy(self.b_org.copy())
                        degenerate = True
                        self.epsilon *= 2
                        break
                    else:
                        print(f'Vertex {self.z} --> Cost: {np.dot(self.z, self.c)}')

                if not degenerate:
                    solved = True

            # Convert the solution back to the original solution, if it exists
            if solution:
                X, y = [], []
                for i in range(len(self.A)):
                    if np.isclose(np.dot(self.A[i], self.z), self.b[i]):
                        X.append(self.A[i])
                        y.append(self.b_org[i])
                X = np.array(X)
                y = np.array(y)
                try:
                    z_solution = np.linalg.solve(X, y)
                except np.linalg.LinAlgError:
                    raise np.linalg.LinAlgError('Final system is singular; cannot find unique solution.')
                print(f'Original Solution: Vertex {z_solution} --> Cost: {np.dot(z_solution, self.c)}')
        except Exception as e:
            print(f'Error: {e}')


In [2]:
if __name__ == '__main__':
    # Replace the filename with the path to your input CSV file
    filename = r'neha.csv'
    solver = SimplexSolver(filename)
    solver.solve()


Vertex [-0.1875  4.25  ] --> Cost: -16.4375
Vertex [0.375 3.125] --> Cost: -13.625
Vertex [ 3.625 -0.125] --> Cost: -10.375
Original Solution: Vertex [ 4. -0.] --> Cost: -12.0
